**Кредитный скоринг** позволяет спрогнозировать вероятность невозврата кредита.  Используя данные о клиенте, работник банка может решить, выдавать ли клиенту кредит, и если да, то под какой процент. При этом используемый в кредитном скоринге алгоритм позволяет предсказывать значения непрерывной зависимой переменной на интервале от 0 до 1. Банки самостоятельно определяют для себя значения, при которых они принимают решение об отказе в кредите.

# Импорт библиотек:

In [ ]:
import os
import sklearn
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm
import catboost as cb
import xgboost as xgb
import optuna
from datetime import datetime
import warnings

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import f_classif, mutual_info_classif

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score,plot_roc_curve
from sklearn.metrics import classification_report, precision_score, recall_score, plot_precision_recall_curve
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm



from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.inspection import permutation_importance

warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
!pip freeze > requirements.txt

RANDOM_SEED = 42

# Загрузка данных:

In [ ]:
data_directory = '/kaggle/input/sf-dst-scoring/'
df_train = pd.read_csv(data_directory+'train.csv')
df_test = pd.read_csv(data_directory+'test.csv')
sample_submission = pd.read_csv(data_directory+'/sample_submission.csv')

In [ ]:
df_train['sample'] = 1  # train
df_test['sample'] = 0  # test

df = df_test.append(df_train, sort=False).reset_index(
    drop=True)  # комбинируем признаки для корректной работы

In [ ]:
df.info()
print()
display(df.sample(3))

# Осмотр данных:

In [ ]:
df_train.columns

**Признаки:**

`client_id` - идентификатор клиента

`education` - уровень образования

`sex` - пол заемщика

`age` - возраст заемщика

`car` - флаг наличия автомобиля

`car_type` - флаг автомобиля иномарки

`decline_app_cnt` - количество отказанных прошлых заявок

`good_work` - флаг наличия “хорошей” работы

`bki_request_cnt` - количество запросов в БКИ

`home_address` - категоризатор домашнего адреса

`work_address` - категоризатор рабочего адреса

`income` - доход заемщика

`foreign_passport` - наличие загранпаспорта

`sna` - связь заемщика с клиентами банка

`first_time` - давность наличия информации о заемщике

`score_bki` - скоринговый балл по данным из БКИ

`region_rating` - рейтинг региона

`app_date` - дата подачи заявки

`default` - флаг дефолта по кредиту

In [ ]:
# Узнаем типы данных наших признаков:

df_agg = df.agg({'nunique', lambda s: s.unique()[:10]})\
    .append(pd.Series(df.isnull().sum(), name='null'))\
    .append(pd.Series(df.dtypes, name='dtype'))\
    .transpose()
df_agg

In [ ]:
df.isna().sum() # Можем видеть пропуски в признаке education.

In [ ]:
#Каких клиентов у банка боььше: с дефолтом или без? 0 - нет, 1 - да.

df.default.hist()
print()
print(df.default.value_counts())

In [ ]:
# Рассмотрим признак education:

print(df['education'].value_counts())
print()
print('Распределение пропусков: ',df['education'].isna().value_counts())

In [ ]:
# Заменим пропуски в education:

df.education = df.education.fillna("SCH")

df['education'].value_counts()

In [ ]:
df['good_work'].value_counts() # Посмотрим на новый признак Good work.

# Вывод по EDA:

- В наборе даных 19 признаков.
- Всего представлено 110 148 клиентов.
- Количество пропусков 0.04%, только в признаке education. 
- Дубликатов нет.
- `client_id` не имеет повторяющихся значений, все значения уникальные, что ставит под сомнения его значимость.
- `app_date` только 120 вариантов признака (0.1%). Большинство данных за период февраль-апрель 2014 года.
- `education` содержит 5 категорий:
    -SCH (52%) - School;
    -GRD (31%) - Graduated (Master degree);
    -UGR (13%) - UnderGraduated (Bachelor degree);
    -PGR (1.7%) - PostGraduated;
    -ACD (0.3%) - Academic Degree.
    -No education (0.1%)
- `sex` содержит 2 вариации признака:
    -Female (56%);
    -Male (44%);
- `age` представлен конкретными значениями со смещением влево: 
    -- Minimum 21 
    -- median 37 
    -- Mean 39.2 
    -- Maximum 72 
    -- Interquartile range (IQR) 18
- `car` бинарный признак, 67% заемщиков не имеют автомобиля.
- `car_type` бинарный признак, показывающий отечественный или иностранный автомобиль у заемщика. 81% заемщиков имеют отечественный автомобиль. 
- `decline_app_cnt` cодержит конкретные значения со смещением влево. Большинство значений (83%) нулевые. Преобладающее большинство наблюдений в промежутке от 0 до 6.
- `good_work` Mбольшинство заемщиков не имеют хорошей работы (83%).
- `score_bki` 93% значений уникальны, распределение нормальное, присутствуют отрицательные значения.
- `bki_request_cnt` натуральные числа, которые варбируются от 0 до 53 с медианой 1. Большинство значений в промежутке от 0 до 8.
- `region_rating` варбируются между 20 и 80. Категориальный признак. Самое часто встречающееся значение 50 (37%).
- `home_address`, work_address категориальные признаки с 3 вариациями.
- `income` большой разброс значений от 1000 до 1000000; можно попробовать либо превратить в категориальный признак, либо прологарифмировать.
- `sna / first_time` категориальные признаки с 4 вариациями значений.
- `foreign_passport` бинарный признак, 67% заемщиков имеют заграничный паспорт.
- `default` целевой признак. Бинарный признак с подавляющим большинством тех, кто возвращает кредит без проблем. Выборка несбалансированная, при моделировании нужно будет попробовать undersampling.

In [ ]:
# Разделим наши признаки на числовые, категориальные и бинарные.

num_cols = [
    'age', 
    'decline_app_cnt', 
    'score_bki',
    'income', 
    'bki_request_cnt'
]

cat_cols = [
    'education', 
    'work_address', 
    'home_address', 
    'region_rating' , 
    'sna', 
    'first_time'
]

bin_cols = [
    'sex', 
    'car', 
    'car_type', 
    'good_work', 
    'foreign_passport'
]

# Обзор колличественных признаков:

In [ ]:
# Посмотрим на распределение количественных признаков
for i in num_cols:
    plt.figure()
    sns.displot(df[i][df[i] > 0].dropna(), kde = False, rug=False)
    plt.title(f'Distribution of {i}')
    plt.show()

In [ ]:
def get_boxplot(df, col):
    fig, axes = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='default', y=col, data=df[df['sample']==1], ax=axes)
    axes.set_title('Boxplot for ' + col)
    plt.show()

# Посмотрим на выбросы и распределение целевой переменной между количественными признаками
for col in num_cols:
    get_boxplot(df, col)

In [ ]:
df.score_bki.hist()

Можно заметить, что только признак `score_bki` имеет почти нормальное распределение. Распределение по возрасту смещено влево. У данных есть выбросы. У нас есть несколько вариантов: 1) Применить функцию логарифма к данным, 2) преобразовать некоторые данные в категориальные признаки, применить комбинацию логарифма и преобразовать в категориальные признаки.

In [ ]:
# Посмотрим статистическую значимость колличественных признаков:

imp_num = pd.Series(f_classif(df_train[num_cols], df_train['default'])[
                    0], index=[num_cols])
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

In [ ]:
def corr_matrix(data, det=True, pltx=10, plty=10):
    '''Funcion is called for making correlation matrix'''
    
    X = data.corr()
    if det:
        
        evals,evec = np.linalg.eig(X)
        ev_product = np.prod(evals)
    
        print(f'Rank of Matrix: {np.linalg.matrix_rank(X)}')
        print(f'Determinant of matrix: {np.round(ev_product,4)}')
        print(f'Shape of matrix: {np.shape(X)}')
    
    plt.figure(figsize=(pltx,plty))
    sns.heatmap(X,vmin=0,vmax=.9,annot=True,square=True)
    plt.show()

corr_matrix(df_train[num_cols])

Наиболее статистически значимыми переменными являются score_bki, reduce_app_cnt. Между функциями нет сильной корреляции. Это подтверждается рангом матрицы.

# Работа с дата-признаками:

In [ ]:
df_train['app_date'] = pd.to_datetime(
    df_train['app_date'])  # Конвертируем нашу дату в формат datetime.

# Функция возвращает порядкой день в году:
def get_days_count(x):  
    day = ((x.month-1) * 30)+x.day
    return day

df_train['days_numb'] = df_train['app_date'].apply(
    lambda x: (get_days_count(x)))  

# Функция, которая возвращает количество дней между датой заказа и сегодняшней датой.
def get_days_beetwen(x):
    curr_date = datetime.today()
    count = (curr_date-x).days
    return count

df_train['days_beetwen'] = df_train['app_date'].apply(
    lambda x: (get_days_beetwen(x)
               )) 

# Функция возвращает месяц:
def month(x):
    month = x.month
    return month

df_train['month'] = df_train['app_date'].apply(lambda x: (month(x)))



In [ ]:
# Удалим признак app_date
df_train = df_train.drop('app_date', axis=1)

In [ ]:
df_train['month'].value_counts() #В нашем датасете только 4 месяца, что довольно немного.

# Обзор бинарных признаков:

In [ ]:
plt.figure(figsize=[30, 30])
i = 1

for k in bin_cols:
    plt.subplot(4, 3, i)
    sns.barplot(x=k,
                y='proportion',
                hue='default',
                data=df_train[[k, 'default']].value_counts(
                    normalize=True).rename('proportion').reset_index())
    
    plt.title('Binary Feature Name\n' + k, fontsize=15)
    i += 1
plt.tight_layout()
plt.show()

- Заемщики-женщины имеют тенденцию к дефолту несколько чаще, чем мужчины.

- Наличие автомобиля и в частности иностранной марки автомобиля делает заемщиков более надежными.

- Люди, имеющие хорошую работу и заграничный паспорт, чаще возвращают ссуды, чем те, у кого плохая работа.

In [ ]:
# Закодируем наши признаки:

mapp = {}
label_encoder = LabelEncoder()
for col in bin_cols:
    df_train[col] = label_encoder.fit_transform(df_train[col])
    mapp[col] = dict(enumerate(label_encoder.classes_))
    
print(mapp)


In [ ]:
# Проверим статистическую значимость:

imp_bol = pd.Series(mutual_info_classif(df_train[bin_cols], df_train['default'],
                                        discrete_features=True), index=[bin_cols])
imp_bol.sort_values(inplace=True)
imp_bol.plot(kind='barh')

In [ ]:
corr_matrix(df_train[bin_cols])

Возможно, стоит объединить признаки `car_type` и `car`.

# Обзор категориальных признаков:

In [ ]:
# Посмотрим на распределение дефолтных состояний по различным признакам

plt.figure(figsize=[20, 20])
i = 1

for k in cat_cols:
    plt.subplot(4, 3, i)
    sns.barplot(x=k, y='proportion', hue='default',  data=df_train[[
                k, 'default']].value_counts(normalize=True).rename('proportion').reset_index())
    plt.title('Binary Feature Name\n' + k, fontsize=15)
    i += 1
plt.tight_layout()
plt.show()

- Люди с низким уровнем образования чаще не возвращают ссуды, чем люди с высоким уровнем образования, которые в свою очередь чаще занимают деньги.
- Категория 4 в системе sna имеет более высокий процент несостоятельных заемщиков. И падает при понижении категории с 4 до 1.
- Столбцы `first_time` также показывают падение доли несостоятельных заемщиков.
- Домашний и рабочий адреса практически совпадают.

In [ ]:
# Выведем зависимость дефолтных/недефолтных клиентов от уровня региона на boxplot графике:

def get_boxplot(data, col1, col2, hue=None):
    
    fig, ax = plt.subplots(figsize=(7, 5))
    sns.boxplot(x=col1, y=col2, hue=hue, data=data)
    plt.xticks(rotation=45)
    ax.set_title(f'Boxplot for {col1} and {col2}', fontsize=14)
    plt.show()


get_boxplot(df_train, 'default', 'region_rating')

Чем выше рейтинг города - тем надежнее заемщик.

In [ ]:
# Посмотрим на связь между уровнем образования и связью заемщиков.

get_boxplot(df_train, 'education', 'sna', hue='default')

Связь есть, но объяснить ее довольно трудно.

In [ ]:
# Заменим наши переменные:

df_train.education = df_train.education.fillna("SCH")

mapp2 = {}
label_encoder = LabelEncoder()
for col in cat_cols:
    df_train[col] = label_encoder.fit_transform(df_train[col])
    mapp2[col] = dict(enumerate(label_encoder.classes_))
    
print(mapp2)

In [ ]:
corr_matrix(df_train[cat_cols])

Можно заметить, что признаки `sna` и `first_time` сильно коррелируют!

In [ ]:
# Оценим матрицу корреляций всех признаков:

corr_matrix(df_train.drop(['sample'], axis=1), det=False, pltx=20, plty=20)

# Модель наивной логистической регрессии:

Построим наивную модель логистической регрессии. Таким образом можно будет оценивать метрики при изменении набора данных и так далее.

In [ ]:
# Разделим датасет:
X_n = df_train.drop(['default'], axis=1)
y_n = df_train['default']

# Создадим трэйн и тест подборку:

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_n,
                                                    y_n,
                                                    test_size=.2,
                                                    random_state=RANDOM_SEED)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_n, y_train_n)
y_pred = lr.predict(X_test_n)
cf_matrix = confusion_matrix(y_test_n, y_pred)

In [ ]:
# Функция для confusion matrix:

def make_confusion_matrix(cf,
                          group_names=['TN', 'FP', 'FN', 'TP'],
                          categories='auto',
                          sum_stats=True,
                          count=True,
                          cbar=True,
                          percent=True,
                          cmap='BuPu'):
    '''Function is called for making a confusion matrix
    args
    ------
    cf - confusion matrix
    group_names - Names for each group
    categories -  categories to be displayed on the x,y axis. Default is 'auto'
    sum_stats -   shows Accuracies. Deafult is TRUE
    c_bar -       If True, show the color bar. The cbar values are based off the values in the confusion matrix. 
                  Default is True
    percent -     to be displayed on the x,y axis. Default is True
    '''
    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    group_labels = ["{}\n".format(value) for value in group_names]

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = [
            "{0:.2%}".format(value) for value in cf.flatten() / np.sum(cf)
        ]
    else:
        group_percentages = blanks

    box_labels = [
        f"{v1}{v2}{v3}".strip()
        for v1, v2, v3 in zip(group_labels, group_counts, group_percentages)
    ]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0], cf.shape[1])

    # Metrics
    if sum_stats:
        # Accuracy is sum of diagonal divided by total observations
        accuracy = np.trace(cf) / float(np.sum(cf))
        ball_accuracy = .5 * (cf[1, 1] / sum(cf[1, :]) +
                              cf[0, 0] / sum(cf[0, :]))

        # if it is a binary confusion matrix, show some more stats
        if len(cf) == 2:
            # pr = how many real true
            precision = cf[1, 1] / sum(cf[:, 1])
            # How many positives from all positives
            recall = cf[1, 1] / sum(cf[1, :])
            # F1 score
            f1_score = 2 * precision * recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nBallancedAcc={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy, ball_accuracy, precision, recall, f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""

    plt.rcParams.get('figure.figsize')
    plt.figure(figsize=(5, 5))
    sns.heatmap(cf,
                annot=box_labels,
                fmt="",
                cmap=cmap,
                cbar=cbar,
                xticklabels=categories,
                yticklabels=categories)
    plt.ylabel('True label')
    plt.xlabel('Predicted label' + stats_text)

In [ ]:
# Функция для ROC-auc:

def make_roc_auc(model, X, y):
    '''Plot ROC-AUC and PR curves'''
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))
    plot_precision_recall_curve(model, X, y, ax=ax1)
    plot_roc_curve(model, X, y, ax=ax2)
    plt.show()

In [ ]:
categories = ['NO DEFAULT', 'DEFAULT']
# Создадим confusion_matrix:
make_confusion_matrix(cf_matrix,categories=categories,sum_stats=True)
# Создадим roc-auc:
make_roc_auc(lr,X_test_n,y_test_n)

- Accuracy=0.876, что неплохо, но наша модель не сбалансирована, мы можем классифицировать только отдельный класс тех, кто вернет деньги. Но наша задача классифицировать другой противоположный класс. 
- Оценка F1 - 0.
- Площадь под кривой (AUC) составляет 0,58. Хотя в идеальном случае значение должно стремиться к 1.
- Площадь под кривой Precision и Reacall очень мала.

# Feature Engineering:

In [ ]:
# Закодируем наши признаки car и car_type:
label_encoder = LabelEncoder()
df_train['car'] = label_encoder.fit_transform(df_train['car'])
df_train['car_type'] = label_encoder.fit_transform(df_train['car_type'])
  
# (Здесь во время выполнения проекта возникла небольшая путаница, поэтому пришлось сделать несколько кодировок.)
df['car'] = label_encoder.fit_transform(df['car'])
df['car_type'] = label_encoder.fit_transform(df['car_type'])
    
# Уменьшим размер. Скомбинируем признаки `car` и `car_type`. Закодируем эти признаки. reduce size.
# 0 - Has no car, 1-Has a good car, 3- has a car
df['car_comb'] = df['car'] + df['car_type']
df = df.drop(['car', 'car_type'], axis=1)
df['car_comb'] = df['car_comb'].astype('category')

In [ ]:
# Добавим четыре возврастные категории (названия используются для простого разделения, это лишь цифра в паспорте!):
def age_to_cat(age):
    if age <= 28:
        cat_age = 'young'
        return cat_age             
    if 28 < age <= 35:
        cat_age = 'not-young'
        return cat_age
    if 35 < age <= 50:
        cat_age = 'middle'
        return cat_age
    if age > 50:
        cat_age = 'old'
        return cat_age

In [ ]:
df['age_cat'] = 0 # Создадим такой признак:
df['age_cat'] = df['age'].apply(lambda x:age_to_cat(x))
df = df.drop('age',axis=1)

df['age_cat'].hist()

Можем заметить, что у нас больше 'middle' клиентов.

In [ ]:
# Подчистим наши признаки 'decline_app_cnt' и 'bki_request_cnt'

df['decline_cat'] = df['decline_app_cnt'].apply(lambda x: 4 if x >= 4 else x) 

df['bki_request_cat'] = df['bki_request_cnt'].apply(lambda x: 7 if x >= 7 else x) # option 1
df = df.drop('decline_app_cnt',axis=1)
df = df.drop('bki_request_cnt',axis=1)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Возьмем две колонки:
data_example = df[['work_address', 'home_address']].values

# Создадим Scaler образец:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_example)

# Получили два вектора. Возьмем только самую важную информацию.
pca = PCA(n_components=1)
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)
df['pca_address'] = pca_data

# Удалим ненужную колонку.
df = df.drop(['home_address','work_address'],axis=1)

In [ ]:
# Конвертируем дату и произведем знакомые операции.
df['app_date'] = pd.to_datetime(
    df['app_date'])

df['days_beetwen'] = df['app_date'].apply(
    lambda x: (get_days_beetwen(x)
               ))

df['month'] = df['app_date'].apply(lambda x: (month(x)))
df = df.drop('app_date', axis=1)

In [ ]:
# Далее в некоторых работах предлагается создать признак "has_no_guarantor" исходя из уровня образования.

def has_no_garant(edu, grnt):
    if edu == 'PGR' or edu == 'ACD':
        grnt = 1
        return grnt
    else:
        grnt = 0
        return grnt
    
    
df['has_no_guarantor'] = 0
df['has_no_guarantor'] = df[['education', 'has_no_guarantor']].apply(
    lambda x: has_no_garant(*x), axis=1)

In [ ]:
df_boost = df.copy()

# Encoding:

In [ ]:
# label Encoder
mapp={}
for i in list(['sex', 'foreign_passport', 'good_work', 'has_no_guarantor']):
    df[i] = label_encoder.fit_transform(df[i])
    mapp[i] = dict(enumerate(label_encoder.classes_))
    
print(mapp)

In [ ]:
class Preprocessing:
    def __init__(self, data):
        self.data = data
    #Напишем функцию по преобразованию категориальных признаков one hot encoder:
    def hot_enc(self, column):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        aux_df = pd.DataFrame(ohe.fit_transform(self.data[[column]]))
        aux_df.columns = ohe.get_feature_names([f'hot_{column}'])
        self.data = self.data.drop(col, axis=1)
        self.data = pd.concat([self.data, aux_df], axis=1)
        return self.data     

In [ ]:
# Конвертируем категориальные признаки:
# one hot encoder

encoder = Preprocessing(df)

cols_to_hot = ['education','region_rating',
               'sna','first_time','car_comb',
               'decline_cat','bki_request_cat',
              'month','age_cat'] 
for col in cols_to_hot:
    df = encoder.hot_enc(col)

In [ ]:
cols_to_drop = ['hot_decline_cat_3','hot_region_rating_30',
                'hot_education_PGR','hot_decline_cat_2',
                'hot_region_rating_20','hot_sna_3',
                'hot_decline_cat_4','has_no_guarantor',
                'hot_bki_request_cat_6','hot_bki_request_cat_5', 
                'hot_bki_request_cat_2','hot_region_rating_60',
                'hot_bki_request_cat_1', 'hot_bki_request_cat_4',
                'hot_age_cat_not-young']
df = df.drop(cols_to_drop, axis=1)

cols_to_drop_b = ['month','foreign_passport','has_no_guarantor']
df_boost = df_boost.drop(cols_to_drop_b, axis=1)

df['pca_address'] = df['pca_address'] + 5

cols_to_log = ['days_beetwen','pca_address','income']
for col in cols_to_log:
    df[col] = df[col].apply(lambda x: np.log(x) +1)
df.sample(3)

# Features scale:

In [ ]:
# Возьмем часть датафрейма для тестинга:
df = df.drop(['client_id'], axis=1)
df_train = df_train.drop(['client_id'], axis=1)
df_train = df.query('sample == 1').drop(['sample'], axis=1)

df_valid = df.query('sample == 0').drop(['sample'], axis=1)

# Возьмем дату и лэйбл.
X = df_train.drop(['default'], axis=1)

y = df_train[['default']]

# Возьмем дату для валидации модели.
X_valid = df_valid.drop(['default'], axis=1)

In [ ]:
scaler = RobustScaler()
cols_to_scal = ['income', 'days_beetwen', 'pca_address']

X[cols_to_scal] = scaler.fit_transform(X[cols_to_scal])
X_valid[cols_to_scal] = scaler.transform(X_valid[cols_to_scal])

In [ ]:
print(
    f'Shape of X_train:{X.shape} \nShape of X_Valid:{X_valid.shape} \nShape of Target:{y.shape} ')

In [ ]:
X_valid

# Наивная лог регрессия с новыми признаками:

`True positive` - корректно классифицированный default клиент.

`False positive` - не default клиент неккоректно классифицирован как default клиент.

`False Negative` - default клиент некорректно классифицирован как не default.

`True negative` - корректно классифицированный не default клиент.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, random_state=RANDOM_SEED)

In [ ]:
# Посмтроим модель:
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Сделаем предсказание:
y_pred = lr.predict(X_test)

# Создадим confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
categories = ['NOT DEFAULT', 'DEFAULT']
make_confusion_matrix(cf_matrix,categories=categories,sum_stats=True)
make_roc_auc(lr,X_test,y_test)


# Modeling:

In [ ]:
def show_metrics(y_test, y_pred, probs):
    print('accuracy_score:\t\t {:.4}'.format(accuracy_score(y_test, y_pred)))
    print('precision_score:\t {:.4}'.format(precision_score(y_test, y_pred, zero_division=0)))
    print('recall_score:\t\t {:.4}'.format(recall_score(y_test, y_pred, zero_division=0)))
    print('f1_score:\t\t {:.4}'.format(f1_score(y_test, y_pred, zero_division=0)))
    print('roc_auc_score:\t\t {:.4}'.format(roc_auc_score(y_test, probs)))
def compute_selected_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    probs = model.predict_proba(X_test)
    probs = probs[:,1]
    show_metrics(y_test, y_pred, probs)
    return y_pred, probs

In [ ]:
# default model

lr = LogisticRegression(max_iter=500)
y_pred, probs = compute_selected_model(lr)

In [ ]:
# penalty=none is stronger than penalty=l2

lr_penalty_none = LogisticRegression(penalty='none', max_iter=1000)
y_pred, probs = compute_selected_model(lr_penalty_none)

In [ ]:
# multi_class=multinominal is weaker than auto or ovr (them equal)

lr_ovr = LogisticRegression(penalty='l2', max_iter=1000, multi_class='ovr')
y_pred, probs = compute_selected_model(lr_ovr)

In [ ]:
# higher max_iter - higher metrics (costs time)

lr_sag = LogisticRegression(penalty='l2', max_iter=1500, solver='sag')
y_pred, probs = compute_selected_model(lr_sag)

In [ ]:
# saga is weaker than sag with equal max_iter
# both sag and saga weaker than default solver

lr_saga = LogisticRegression(penalty='l2', max_iter=1500, solver='saga')
y_pred, probs = compute_selected_model(lr_saga)

In [ ]:
# balanced is weaker than default settings in roc_auc
# but f1_score is significant stronger

lr_balanced = LogisticRegression(class_weight='balanced', max_iter=500)
y_pred, probs = compute_selected_model(lr_balanced)

- Настройка сбалансированного веса классов ухудшает метрику ROC AUC по сравнению с настройками по умолчанию (f1_score, при этом, улучшается существенно).
- Solvers sag, saga - слабее, чем solver по умолчанию (lbfgs).
- На заданном наборе данных penalty=none эффективнее penalty=l2.
- Мультикласс-настройка multinominal формирует более слабую модель. Ovr формирует идентичную модель с настройкой по умолчанию - auto.

# Оценка ROC AUC и других метрик:

In [ ]:
# best LogReg model from previous chapter with balanced class weights

lr_balanced_penalty_none = LogisticRegression(class_weight='balanced', penalty='none', max_iter=1000)
y_pred, probs = compute_selected_model(lr_balanced_penalty_none)

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

- Настройка сбалансированного веса классов ухудшает метрику ROC AUC по сравнению с настройками по умолчанию (f1_score, при этом, улучшается существенно).
- Solvers sag, saga - слабее, чем solver по умолчанию (lbfgs).
- На заданном наборе данных penalty=none эффективнее penalty=l2.
- Мультикласс-настройка multinominal формирует более слабую модель. Ovr формирует идентичную модель с настройкой по умолчанию - auto.

# Подбор гиперпараметров:

In [ ]:
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

X_train = train_data.drop(['default'], axis=1)
y_train = train_data.default.values
X_test = test_data.drop(['default'], axis=1)
y_test = test_data.default.values

In [ ]:
# Penalty - Used to specify the norm used in the penalization

# C - Inverse of regularization strength; 
# must be a positive float.
# Like in support vector machines, smaller values specify stronger regularization.

# solver - Algorithm to use in the optimization problem.
# UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'

params = {
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs', 'liblinear', 'sag', 'saga']
}

model = LogisticRegression(penalty='l2', max_iter=1000, class_weight='balanced', n_jobs=4)
model.fit(X_train, y_train)

clf = GridSearchCV(model, params, cv=5, verbose=3)

best_model = clf.fit(X_train, y_train)

print('Лучшее C:', best_model.best_estimator_.get_params()['C'])
print('Лучшее solver:', best_model.best_estimator_.get_params()['solver'])

In [ ]:
y_pred = best_model.predict_proba(X_test)
results_df = pd.DataFrame(data={'client_id':df_test['client_id'], 'default':y_pred[:,1]})
results_df.to_csv('submission.csv', index=False)
results_df

**В итоговый вариант работы вошел вариант с подбором гиперпараметров для logisticRegression со сбалансированными признаками - несмотря на чуть более низкий ROC AUC, такая модель демонстрирует более качественные результаты по прочим метрикам - accuracy, precision, recall и, особенно, f1_score.**